In [ ]:
#!pip install pandas google-cloud-bigquery ipython-sql
#!pip install db-dtypes
#!pip install google-cloud-bigquery-storage


In [2]:
import os
from google.cloud import bigquery
import pandas as pd
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "cmpe180b-project-f6eb9de16408.json"



## Fetch all table names

In [35]:

# Initialize the client
client = bigquery.Client()

# Replace with your project and dataset ID
project_id = "bigquery-public-data"
dataset_id = "nhtsa_traffic_fatalities"

# Get a reference to the dataset
dataset_ref = client.dataset(dataset_id, project=project_id)

# List all tables in the dataset
tables = client.list_tables(dataset_ref)

# Print all table names
#for table in tables:
#    print(table.table_id)

table_list = [table.table_id for table in tables]
df_tables = pd.DataFrame(table_list, columns=["table_name"])
# Option 2: Using pd.option_context (temporary change)

# We only want the first 103 tables, because the rest are repeated
df_tables = df_tables[:103]

df_tables["table_id"] = "bigquery-public-data.nhtsa_traffic_fatalities. " + df_tables["table_name"]

with pd.option_context('display.max_rows', None, 'display.max_columns', None, 'display.width', 0):
    print(df_tables)
#df_tables.to_csv("nhtsa_tables.csv", index=False)



          table_name                                           table_id
0      accident_2015  bigquery-public-data.nhtsa_traffic_fatalities....
1      accident_2016  bigquery-public-data.nhtsa_traffic_fatalities....
2      accident_2017  bigquery-public-data.nhtsa_traffic_fatalities....
3      accident_2018  bigquery-public-data.nhtsa_traffic_fatalities....
4      accident_2019  bigquery-public-data.nhtsa_traffic_fatalities....
5      accident_2020  bigquery-public-data.nhtsa_traffic_fatalities....
6        cevent_2015  bigquery-public-data.nhtsa_traffic_fatalities....
7        cevent_2016  bigquery-public-data.nhtsa_traffic_fatalities....
8        cevent_2017  bigquery-public-data.nhtsa_traffic_fatalities....
9        cevent_2018  bigquery-public-data.nhtsa_traffic_fatalities....
10       cevent_2019  bigquery-public-data.nhtsa_traffic_fatalities....
11       cevent_2020  bigquery-public-data.nhtsa_traffic_fatalities....
12       damage_2015  bigquery-public-data.nhtsa_traffic_fatalit

# Get the schema of all tables

In [36]:
# Initialize client
client = bigquery.Client()

# Store all schema details
schema_rows = []

# Loop through all 2015 tables in df_tables
for table_name in df_tables["table_name"]:
    full_table_id = f"bigquery-public-data.nhtsa_traffic_fatalities.{table_name}"
    
    try:
        table = client.get_table(full_table_id)
        for field in table.schema:
            schema_rows.append({
                "table_name": table_name,
                "column_name": field.name,
                "data_type": field.field_type
            })
    except Exception as e:
        print(f"Failed to get schema for {table_name}: {e}")

# Create a DataFrame with all schema info
df_schema_all = pd.DataFrame(schema_rows)

# Preview the result
# with pd.option_context('display.max_rows', None, 'display.max_columns', None, 'display.width', 0):
    # print(df_schema_all)
#df_schema_all.to_csv("nhtsa_2015_table_schemas.csv", index=False)


In [50]:
from google.cloud import bigquery
import pandas as pd

client = bigquery.Client()

# Step 1: List all tables in the dataset
dataset_id = "bigquery-public-data.nhtsa_traffic_fatalities"
tables = client.list_tables(dataset_id)

# Step 2: Build DataFrame of table names and full IDs
table_data = []
table_query_dict = {}
for table in tables:
    table_name = table.table_id
    if table_name.startswith(" cevent"):  # leading space is intentional!
        full_table_id = f"{dataset_id}.{table_name}"
        table_data.append({"table_name": table_name, "table_id": full_table_id})

df_tables = pd.DataFrame(table_data)

# Function to fetch column names for a table
def get_columns(table_id):
    schema = client.get_table(table_id).schema
    return [field.name for field in schema]

# Step 3: Collect all column names across the accident tables
all_columns = set()
table_columns = {}

for _, row in df_tables.iterrows():
    table_id = row["table_id"]
    try:
        cols = get_columns(table_id)
        table_columns[table_id] = cols
        all_columns.update(cols)
    except Exception as e:
        print(f"Error fetching schema for {table_id}: {e}")

all_columns = sorted(all_columns)

# Step 4: Construct SELECT statements for each table
select_statements = []

for table_id, cols in table_columns.items():
    select_parts = []
    for col in all_columns:
        if col in cols:
            select_parts.append(f"`{col}`")
        else:
            select_parts.append(f"NULL AS `{col}`")
    select_query = f"SELECT {', '.join(select_parts)} FROM `{table_id}`"
    select_statements.append(select_query)

# Step 5: Combine all SELECTs into a view definition
if select_statements:
    view_sql = f"""
    CREATE OR REPLACE VIEW `your_dataset.accident_all_years` AS
    {chr(10).join(['\nUNION ALL\n'.join(select_statements)])}
    """
    print(view_sql.strip())
else:
    print("⚠️ No accident tables with leading space found.")


CREATE OR REPLACE VIEW `your_dataset.accident_all_years` AS
    SELECT `area_of_impact_other_vehicle`, `area_of_impact_other_vehicle_name`, `area_of_impact_this_vehicle`, `area_of_impact_this_vehicle_name`, `consecutive_number`, `event_number`, `sequence_of_events`, `sequence_of_events_name`, `state_name`, `state_number`, `vehicle_number_other_vehicle`, `vehicle_number_other_vehicle_name`, `vehicle_number_this_vehicle` FROM `bigquery-public-data.nhtsa_traffic_fatalities. cevent_2015`
UNION ALL
SELECT `area_of_impact_other_vehicle`, `area_of_impact_other_vehicle_name`, `area_of_impact_this_vehicle`, `area_of_impact_this_vehicle_name`, `consecutive_number`, `event_number`, `sequence_of_events`, `sequence_of_events_name`, `state_name`, `state_number`, `vehicle_number_other_vehicle`, `vehicle_number_other_vehicle_name`, `vehicle_number_this_vehicle` FROM `bigquery-public-data.nhtsa_traffic_fatalities. cevent_2016`
UNION ALL
SELECT `area_of_impact_other_vehicle`, `area_of_impact_other_vehic

In [51]:
from google.cloud import bigquery
import pandas as pd
import re

client = bigquery.Client()

# Step 1: List all tables in the dataset
dataset_id = "bigquery-public-data.nhtsa_traffic_fatalities"
tables = client.list_tables(dataset_id)

# Step 2: Build DataFrame of table names and full IDs (preserve leading spaces!)
table_data = []
for table in tables:
    table_name = table.table_id
    full_table_id = f"{dataset_id}.{table_name}"
    table_data.append({"table_name": table_name, "table_id": full_table_id})

df_tables = pd.DataFrame(table_data)

# Step 3: Extract prefix (handles leading spaces)
df_tables["prefix"] = df_tables["table_name"].str.extract(r"^\s*([a-zA-Z]+)")

# Function to fetch columns from a table
def get_columns(table_id):
    schema = client.get_table(table_id).schema
    return [field.name for field in schema]

# Step 4: Group tables by prefix
grouped = df_tables.groupby("prefix")

# Step 5: Build SQL views per group
sql_views = {}

for prefix, group in grouped:
    all_columns = set()
    table_columns = {}

    # Gather all columns from all tables in this group
    for _, row in group.iterrows():
        table_id = row["table_id"]
        try:
            columns = get_columns(table_id)
            table_columns[table_id] = columns
            all_columns.update(columns)
        except Exception as e:
            print(f"Error fetching schema for {table_id}: {e}")

    all_columns = sorted(all_columns)

    # Build SELECT statements for each table
    select_statements = []
    for table_id, cols in table_columns.items():
        select_parts = []
        for col in all_columns:
            if col in cols:
                select_parts.append(f"`{col}`")
            else:
                select_parts.append(f"NULL AS `{col}`")
        select_query = f"SELECT {', '.join(select_parts)} FROM `{table_id}`"
        select_statements.append(select_query)

    if select_statements:
        # Join queries with UNION ALL
        union_sql = "\nUNION ALL\n".join(select_statements)
        view_sql = f"""
        CREATE OR REPLACE VIEW `your_dataset.{prefix}_all_years` AS
        {union_sql}
        """
        sql_views[prefix] = view_sql.strip()

# Print all view creation SQLs
for prefix, view_sql in sql_views.items():
    print(f"\n-- View for {prefix}_all_years --\n")
    print(view_sql)



-- View for accident_all_years --

CREATE OR REPLACE VIEW `your_dataset.accident_all_years` AS
        SELECT `atmospheric_conditions`, `atmospheric_conditions_1`, `atmospheric_conditions_1_name`, `atmospheric_conditions_2`, `atmospheric_conditions_2_name`, `atmospheric_conditions_name`, `city`, NULL AS `city_name`, `consecutive_number`, `county`, NULL AS `county_name`, `day_name`, `day_of_crash`, `day_of_week`, `day_of_week_name`, `first_harmful_event`, `first_harmful_event_name`, `functional_system`, `functional_system_name`, `hour_of_arrival_at_scene`, `hour_of_arrival_at_scene_name`, `hour_of_crash`, `hour_of_crash_name`, `hour_of_ems_arrival_at_hospital`, `hour_of_ems_arrival_at_hospital_name`, `hour_of_notification`, `hour_of_notification_name`, `land_use`, `land_use_name`, `latitude`, `latitude_name`, `light_condition`, `light_condition_name`, `longitude`, `longitude_name`, `manner_of_collision`, `manner_of_collision_name`, `milepoint`, `milepoint_name`, `minute_of_arrival_at_s

## Fetch all attributes of accident_2015

In [17]:

# Initialize client
client = bigquery.Client()

# Full table ID in the format: project.dataset.table
acc_2015_id = "bigquery-public-data.nhtsa_traffic_fatalities. accident_2015"
acc_2016_id = "bigquery-public-data.nhtsa_traffic_fatalities. accident_2016"
acc_2017_id = "bigquery-public-data.nhtsa_traffic_fatalities. accident_2017"
acc_2018_id = "bigquery-public-data.nhtsa_traffic_fatalities. accident_2018"
acc_2019_id = "bigquery-public-data.nhtsa_traffic_fatalities. accident_2019"
acc_2020_id = "bigquery-public-data.nhtsa_traffic_fatalities. accident_2020"

# Get table metadata
acc_2015 = client.get_table(acc_2015_id)
acc_2016 = client.get_table(acc_2016_id)
acc_2017 = client.get_table(acc_2017_id)
acc_2018 = client.get_table(acc_2018_id)
acc_2019 = client.get_table(acc_2019_id)
acc_2020 = client.get_table(acc_2020_id)

print("Table: Accident_2015")
print(f"Rows: {acc_2015.num_rows}")
print(f"Columns: {len(acc_2015.schema)}")

print("Table: Accident_2016")
print(f"Rows: {acc_2016.num_rows}")
print(f"Columns: {len(acc_2016.schema)}")

print("Table: Accident_2017")
print(f"Rows: {acc_2017.num_rows}")
print(f"Columns: {len(acc_2017.schema)}")

print("Table: Accident_2018")
print(f"Rows: {acc_2018.num_rows}")
print(f"Columns: {len(acc_2018.schema)}")

print("Table: Accident_2019")
print(f"Rows: {acc_2019.num_rows}")
print(f"Columns: {len(acc_2019.schema)}")

print("Table: Accident_2020")
print(f"Rows: {acc_2020.num_rows}")
print(f"Columns: {len(acc_2020.schema)}")
# Print column names and data types
# for schema_field in acc_2015.schema:
#     print(f"{schema_field.name}: {schema_field.field_type}")
    


Table: Accident_2015
Rows: 32287
Columns: 90
Table: Accident_2016
Rows: 34494
Columns: 92
Table: Accident_2017
Rows: 34299
Columns: 92
Table: Accident_2018
Rows: 33692
Columns: 92
Table: Accident_2019
Rows: 33242
Columns: 92
Table: Accident_2020
Rows: 35451
Columns: 82


1. Tracking Fatal Crash Trends over time (yearly, monthly, hourly)

In [6]:
client = bigquery.Client()

# Example query
query = """
SELECT
  EXTRACT(YEAR FROM crash_date) AS year,
  COUNT(*) AS fatal_crashes,
  SUM(fatals) AS total_fatalities
FROM
  `bigquery-public-data.nhtsa_traffic_fatalities. accident_2015` accidents
WHERE
  fatals > 0
GROUP BY
  year
ORDER BY
  year;
"""

df = client.query(query).to_dataframe()
df.head()




BadRequest: 400 Unrecognized name: fatals at [9:3]; reason: invalidQuery, location: query, message: Unrecognized name: fatals at [9:3]

Location: US
Job ID: 68a4baae-5bee-47c7-a008-73010cea1d34
